In [ ]:
import drjit as dr
import mitsuba as mi
mi.set_variant('cuda_ad_rgb')
import json
import math
import numpy as np
import os

In [ ]:
from mitsuba import ScalarTransform4f as T

In [ ]:
exp = "glass_cat"
train_dir = "../data/{}/train".format(exp)
if not os.path.exists(train_dir):
    os.makedirs(train_dir)
    print("The data directory is created!")

img_train_dir = train_dir+"/image"
if not os.path.exists(img_train_dir):
    os.makedirs(img_train_dir)
    print("The image directory is created!")

mask_train_dir = train_dir+"/mask"
if not os.path.exists(mask_train_dir):
    os.makedirs(mask_train_dir)
    print("The mask directory is created!")

In [ ]:
test_dir = "../data/{}/test".format(exp)
if not os.path.exists(test_dir):
    os.makedirs(test_dir)
    print("The data directory is created!")

img_test_dir = test_dir+"/image"
if not os.path.exists(img_test_dir):
    os.makedirs(img_test_dir)
    print("The image directory is created!")

mask_test_dir = test_dir+"/mask"
if not os.path.exists(mask_test_dir):
    os.makedirs(mask_test_dir)
    print("The mask directory is created!")

In [ ]:
def load_sensor_even(origin):
    # Apply two rotations to convert from spherical coordinates to world 3D coordinates.
    origin = origin

    return mi.load_dict({
        'type': 'perspective',
        'fov': 35,
        'to_world': T.look_at(
            origin=origin,
            target=[0, 0, 0],
            up=[0, 1, 0]
        ),
        'sampler': {
            'type': 'independent',
            'sample_count': 256
        },
        'film': {
            'type': 'hdrfilm',
            'width': 512,
            'height': 512,
            'rfilter': {
                'type': 'tent',
            },
            'pixel_format': 'rgba',
        },
    })

In [ ]:
def fibonacci_sphere(samples, rad=2.3):

    points = []
    phi = math.pi * (3. - math.sqrt(5.))  # golden angle in radians

    for i in range(samples):
        y = 1 - (i / float(samples - 1)) * 2  # y goes from 1 to -1
        radius = math.sqrt(1 - y * y)  # radius at y

        theta = phi * i  # golden angle increment

        x = math.cos(theta) * radius
        z = math.sin(theta) * radius
        
        p = [x, y, z]

        points.append(p/np.linalg.norm(p)*rad)

    return points

test_sample = 350
train_sample = 160

origins = fibonacci_sphere(test_sample)[1:201]
import random
random.shuffle(origins)
test_origins = origins[:100]
train_origins = origins[100:200]


In [ ]:
train_sensors = []
for origin in train_origins:
    train_sensors.append(load_sensor_even(origin))
print("number of test sensors: "+str(len(train_sensors)))

In [ ]:
train_sensors[0]

In [ ]:
test_sensors = []
for origin in test_origins:
    test_sensors.append(load_sensor_even(origin))

print("number of test sensors: "+str(len(test_sensors)))

In [ ]:
## Change here to for alternative shape and envmap
scene_mask = mi.load_dict({
    'type': 'scene',
    # The keys below correspond to object IDs and can be chosen arbitrarily
    'integrator': {'type': 'direct',
    'hide_emitters': True,
    },
    'emitter': {
        'type': 'envmap',
        'filename': '../scenes/textures/kitchen_morning.exr'},
    'kitty': {
        'type': 'obj',
        'filename': '../scenes/meshes/cat.obj',
        'to_world': T.scale([.1, .1, .1]),
        'bsdf': {
            'type': 'dielectric',
            'int_ior': 1.4723,
            'ext_ior': 'air'
        },
    },
})

ref_train_masks = [mi.render(scene_mask, sensor=sensor) for sensor in train_sensors]
ref_test_masks = [mi.render(scene_mask, sensor=sensor) for sensor in test_sensors]
print("number of train masks " + str(len(ref_train_masks)))

In [ ]:
# from PIL import Image
import cv2
import numpy as np
import skimage.exposure


In [ ]:
def three_dig(j):
    f = str(j)
    if len(f) == 1:
        return "00"+f
    elif len(f) == 2:
        return "0"+f
    else:
        return f

In [ ]:
for i in range(len(ref_train_masks)):
    bmp = mi.Bitmap(ref_train_masks[i])
    bmp_np = np.array(bmp)
    img_section = bmp_np[:,:,3] > .5
    img_section = img_section.astype(np.uint8) * 255

    # blur threshold image
    blur = cv2.GaussianBlur(img_section, (0,0), sigmaX=1, sigmaY=1, borderType = cv2.BORDER_DEFAULT)
    result = skimage.exposure.rescale_intensity(blur, in_range=(127.5,255), out_range=(0,255))
    # save output
    cv2.imwrite(os.path.join(mask_train_dir, "ref_{}.png".format(three_dig(i))), result)

In [ ]:
for i in range(len(ref_test_masks)):
    bmp = mi.Bitmap(ref_test_masks[i])
    bmp_np = np.array(bmp)
    img_section = bmp_np[:,:,3] > .5
    img_section = img_section.astype(np.uint8) * 255

    blur = cv2.GaussianBlur(img_section, (0,0), sigmaX=1, sigmaY=1, borderType = cv2.BORDER_DEFAULT)
    result = skimage.exposure.rescale_intensity(blur, in_range=(127.5,255), out_range=(0,255))
    # save output
    cv2.imwrite(os.path.join(mask_test_dir, "ref_{}.png".format(three_dig(i+100))), result)

In [ ]:
scene = mi.load_dict({
    'type': 'scene',
    # The keys below correspond to object IDs and can be chosen arbitrarily
    'integrator': {
        'type': 'path',
        # 'hide_emitters': True,
        },
    'emitter': {
        'type': 'envmap',
        'filename': '../scenes/textures/envmap2.exr'},
    'kitty': {
        'type': 'obj',
        'filename': '../scenes/meshes/cat.obj',
        'to_world': T.scale([.1, .1, .1]),
        'bsdf': {
            'type': 'dielectric',
            'int_ior': 1.4723,
            'ext_ior': 'air'
        },
    },
})

ref_train_images = [mi.render(scene, sensor=sensor) for sensor in train_sensors]
ref_test_images = [mi.render(scene, sensor=sensor) for sensor in test_sensors]

print("number of train images " + str(len(ref_train_images)))
print("number of test images " + str(len(ref_test_images)))

In [ ]:
for i in range(len(ref_train_images)):
    mi.util.write_bitmap(os.path.join(img_train_dir, "ref_{}.png".format(three_dig(i))), ref_train_images[i], write_async=False)

In [ ]:
for i in range(len(ref_test_images)):
    mi.util.write_bitmap(os.path.join(img_test_dir, "ref_{}.png".format(i+100)), ref_test_images[i], write_async=False)

In [ ]:
def take_c2w(cam_desc):
    b = str(cam_desc).split('\n')[23:]
    c = np.array(eval((b[0] + b[1] + b[2]+ b[3] + b[4]).replace(" ", "")[9:-1])).flatten()
    c[4:12] *= -1
    return c.tolist()

def cam_intr(cam_desc):
    a = str(cam_desc).split('\n')[1].replace(" ", "")
    fov_rad = int(a[7:9])/2 * math.pi / 180
    focal_length = 256 / math.tan(fov_rad)
    return [focal_length, 0.0, 256.0, 0.0, 0.0, focal_length, 256.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0]

In [ ]:
cam_intrinsic = cam_intr(test_sensors[0])

In [ ]:
camera_dict= {}

cam_intrinsic = cam_intr(train_sensors[0])

for i in range(len(train_sensors)):
    k = three_dig(i)
    filename = "ref_{}.exr".format(k)
    cam_i = {}
    cam_i["K"] = cam_intrinsic
    cam_i["C2W"] = take_c2w(train_sensors[i])
    cam_i["img_size"] = [
      512,
      512
    ]
    camera_dict[filename] = cam_i


with open(os.path.join(train_dir, 'cam_dict_norm.json'), 'w') as outfile:
    json.dump(camera_dict, outfile, indent=4)

In [ ]:
camera_test_dict= {}

for i in range(len(test_sensors)):
    k = three_dig(i+100)
    filename = "ref_{}.exr".format(k)
    cam_i = {}
    cam_i["K"] = cam_intrinsic
    cam_i["C2W"] = take_c2w(test_sensors[i])
    cam_i["img_size"] = [
      512,
      512
    ]
    camera_test_dict[filename] = cam_i


with open(os.path.join(test_dir, 'cam_dict_norm.json'), 'w') as outfile:
    json.dump(camera_test_dict, outfile, indent=4)
